In [82]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
from toolz import *
import seaborn as sns
import graphviz as gr
from graphviz import Digraph
from linearmodels.iv import IV2SLS
from statistics import mean

# 1. Randomly Assigned treatment with covariates

a coinflip is done in order to decide whether a low carb diet is assigned to a patient

modeling randomly assigned low carb diet to patients to determine if they lose weight. covariates are age of patients. 

weightloss = a0 + a1Age + a2LowCarb + e

LowCarb = 0 when treatment is not assigned; LowCarb = 1 when treatment is assigned

### DAG

In [5]:
g = gr.Digraph()
g.edge("coin flip","lowcarb")
g.edge("lowcarb","weightloss")
g.edge("age","weightloss")
#g

### DGP

generating a dataframe with columns for age, low carb, and weight. I decided that the mean age should be 45 years old and the mean weight should be 180 lbs. 

In [6]:
n=100
np.random.seed(1234)

age = {'Age':np.random.normal(45,20,n).round()} # mean age = 45

lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=0,0,1)} #Treatment;dummy only taking 0 or 1

weightloss = {'Weight':np.random.normal(180,65,n).round(2)} #outcome

Age = pd.DataFrame(data=age)
LowCarb = pd.DataFrame(data=lowcarb)
Weightloss = pd.DataFrame(data=weightloss)

df1 = pd.concat([Age,LowCarb,Weightloss],axis=1)
df1 #our data for this

,Age,Low Carb,Weight
0,54.0,1,159.23
1,21.0,1,139.70
2,74.0,1,190.20
3,39.0,1,142.86
4,31.0,1,248.75
...,...,...,...
95,43.0,0,267.31
96,38.0,0,176.71
97,56.0,1,156.34
98,24.0,1,79.03


### Monte Carlo Experiment

In [7]:
data = []

def monte_carlo(n):
    for i in range(n):
        age = {'Age':np.random.normal(45,20,n).round()}

        lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=0,0,1)}
        
        weightloss = {'Weight':np.random.normal(180,65,n).round(2)}
        
        Age = pd.DataFrame(data=age)
        LowCarb = pd.DataFrame(data=lowcarb)
        Weightloss = pd.DataFrame(data=weightloss)
    
        data = pd.concat([Age,LowCarb,Weightloss],axis=1)
    return data
Data1 = monte_carlo(100)
print(Data1.head())
Data2 = monte_carlo(100)
print (Data2.head())

    Age  Low Carb  Weight
0  17.0         1  119.85
1  50.0         1  189.09
2  21.0         0  108.71
3  14.0         1  270.50
4  63.0         1  109.33
    Age  Low Carb  Weight
0  39.0         1  206.29
1  42.0         0  201.97
2  49.0         1  221.69
3  35.0         1   72.23
4  39.0         1  159.13


RMSE Calculation

In [8]:
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

actual = Data1
pred = Data2
RMSE = sqrt(mse(actual,pred))
print(RMSE)

48.06837993941547


Treatment Effect & Bias

Running an OLS regression using the data from the Data1 dataframe 

In [9]:
# Run OLS regression on Data1
ols = smf.ols(formula="Weight ~ LowCarb + Age",data=Data1).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.5980
Date:                Wed, 13 Apr 2022   Prob (F-statistic):              0.552
Time:                        11:06:40   Log-Likelihood:                -550.25
No. Observations:                 100   AIC:                             1107.
Df Residuals:                      97   BIC:                             1114.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    181.6712     16.348     11.113      0.0

the treatment effect can be interpreted as the prediction for the coefficient of LowCarb. An assignment of a low carb diet increases/decreases (depending of if coef is pos/neg) weight by (value of coef for LowCarb). 

#### Bias

Bias for the outcome of treatment being calculated. 
Taking the predicted value for Weight from the OLS regression subtracting the average of the actual Weight from the generated data. 

In [81]:
Predicted_Weight = ols.predict()
Actual_Weight = Data1["Weight"]
Bias = mean(Predicted_Weight) - mean(Actual_Weight)
print(Bias)

-3.3943000000000154


## for uncontrolled

uncontrolled means there will be no inclusion of age as a factor

#### DAG

In [84]:
g = gr.Digraph()
g.edge("coin flip","lowcarb")
g.edge("lowcarb","weightloss")

#### DGP

creating a dataframe with columns for low carb diet assignment and weight

In [ ]:
n=100
np.random.seed(1234)

lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=0,0,1)} #Treatment;dummy only taking 0 or 1

weightloss = {'Weight':np.random.normal(180,65,n).round(2)} #outcome

LowCarb = pd.DataFrame(data=lowcarb)
Weightloss = pd.DataFrame(data=weightloss)

df1 = pd.concat([LowCarb,Weightloss],axis=1)
df1 #our data for this

#### Monte Carlo

In [30]:
data = []

def monte_carlo(n):
    for i in range(n):

        lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=0,0,1)}
        
        weightloss = {'Weight':np.random.normal(180,65,n).round(2)}
        
        LowCarb = pd.DataFrame(data=lowcarb)
        Weightloss = pd.DataFrame(data=weightloss)
    
        data = pd.concat([LowCarb,Weightloss],axis=1)
    return data
Data1 = monte_carlo(100)
print(Data1.head())
Data2 = monte_carlo(100)
print (Data2.head())

   Low Carb  Weight
0         1  244.00
1         0  207.43
2         1  182.31
3         0  137.57
4         0  155.67
   Low Carb  Weight
0         1  106.67
1         0  243.98
2         0   83.95
3         0  209.32
4         1  218.17


RMSE calculation

In [31]:
actual = Data1
pred = Data2
RMSE = sqrt(mse(actual,pred))
print(RMSE)

63.40619910939308


Treatment Effect

In [32]:
# Run OLS regression on Data1
ols = smf.ols(formula="Weight ~ LowCarb",data=Data1).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.3572
Date:                Wed, 13 Apr 2022   Prob (F-statistic):              0.551
Time:                        11:14:48   Log-Likelihood:                -546.65
No. Observations:                 100   AIC:                             1097.
Df Residuals:                      98   BIC:                             1103.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    179.6019      8.348     21.514      0.0

#### Bias

In [79]:
Predicted_Weight = ols.predict()
Actual_Weight = Data1["Weight"]
Bias = mean(Predicted_Weight) - mean(Actual_Weight)
print(Bias)

-3.3943000000000154




# 2. Confounding Variables

Confounder Scenario:

if a patient is considered to have poor self control with their health/lifestyle, they are put on a low carb diet in order to help them lose weight 

modeling again the effect of a low carb diet on a patients weight, but this time the patient is only assigned a low carb diet if they are considered to have poor self control over their health and lifestyle; SelfControl is our confounding variable

if SelfControl = 1, LowCarb = 1; if SelfControl = 0, LowCarb = 0

### DAG

In [87]:
g = gr.Digraph()
g.edge("self control","low carb")
g.edge("self control","weight")
g.edge("low carb","weight")
#g

### DGP

creating dataframe with columns for whether a doctor would consider the patient to have self control (binary), low carb diet assignment, and weight

In [93]:
n=100
np.random.seed(1234)

#Confounder;dummy 
selfcontrol = {'Self Control':np.where(np.random.normal(1,10,n)<=0,0,1)} 

#Treatment;dummy
lowcarb = selfcontrol

weightloss = {'Weight':np.random.normal(230,10,n).round(2)} #outcome


SelfControl = pd.DataFrame(data=selfcontrol)
LowCarb = pd.DataFrame(data=lowcarb)
Weightloss = pd.DataFrame(data=weightloss)

df2 = pd.concat([LowCarb,SelfControl,Weightloss],axis=1)
df2.columns = ['Low Carb','Self Control','Weight'] 
df2 #our data for this


,Low Carb,Self Control,Weight
0,1,1,232.91
1,0,0,235.67
2,1,1,235.04
3,0,0,232.85
4,0,0,234.84
...,...,...,...
95,1,1,225.68
96,0,0,228.39
97,1,1,238.89
98,0,0,232.88


### Monte Carlo Sim

In [35]:
data = []

def monte_carlo(n):
    for i in range(n):
        selfcontrol = {'Self Control':np.where(np.random.normal(1,10,n)<=0,0,1)} 

        lowcarb = selfcontrol
        
        weightloss = {'Weight':np.random.normal(180,65,n).round(2)}
        
        SelfControl = pd.DataFrame(data=selfcontrol)
        LowCarb = pd.DataFrame(data=lowcarb)
        Weightloss = pd.DataFrame(data=weightloss)
    
        data = pd.concat([SelfControl,LowCarb,Weightloss],axis=1)
        data.columns = ['Low Carb','Self Control','Weight']
    return data
DataConfound1 = monte_carlo(100)
print(DataConfound1.head())
DataConfound2 = monte_carlo(100)
print (DataConfound2.head())

   Low Carb  Self Control  Weight
0         0             0  180.90
1         0             0  212.96
2         0             0  144.45
3         1             1  203.77
4         0             0  202.01
   Low Carb  Self Control  Weight
0         0             0  131.03
1         1             1  132.51
2         1             1  159.05
3         0             0  197.04
4         1             1  256.17


RMSE Calculation

In [36]:
actual = DataConfound1
pred = DataConfound2
RMSE = sqrt(mse(actual,pred))
print(RMSE)

51.10953378447769


In [37]:
# Run OLS regression on DataConfound1
ols = smf.ols(formula="Weight ~ LowCarb + SelfControl",data=DataConfound1).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.049
Method:                 Least Squares   F-statistic:                     6.073
Date:                Wed, 13 Apr 2022   Prob (F-statistic):             0.0155
Time:                        11:15:23   Log-Likelihood:                -558.08
No. Observations:                 100   AIC:                             1120.
Df Residuals:                      98   BIC:                             1125.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     192.1793     10.252     18.745      

####  Bias

In [78]:
Predicted_Weight = ols.predict()
Actual_Weight = DataConfound1["Weight"]
Bias = mean(Predicted_Weight) - mean(Actual_Weight)
print(Bias)

-2.842170943040401e-14


### uncontrolled

#### DAG

In [89]:
g = gr.Digraph()
g.edge("low carb","weight")
#g



#### DGP

not controlling for whether or not a patient is considered to have self control, the dataframe now only consists of Low Carb assignment and weight. Here, avg weight is 230 lbs 

In [45]:
n=100
np.random.seed(1234)

lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=0,0,1)}

weightloss = {'Weight':np.random.normal(230,10,n).round(2)} #outcome

LowCarb = pd.DataFrame(data=lowcarb)
Weightloss = pd.DataFrame(data=weightloss)

df2 = pd.concat([LowCarb,Weightloss],axis=1)
df2.columns = ['Low Carb','Weight'] 
df2 #our data for this

,Low Carb,Weight
0,1,232.91
1,0,235.67
2,1,235.04
3,0,232.85
4,0,234.84
...,...,...
95,1,225.68
96,0,228.39
97,1,238.89
98,0,232.88


#### Monte Carlo

In [46]:
data = []

def monte_carlo(n):
    for i in range(n):
       
        lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=0,0,1)}
        
        weightloss = {'Weight':np.random.normal(180,65,n).round(2)}
        
        LowCarb = pd.DataFrame(data=lowcarb)
        Weightloss = pd.DataFrame(data=weightloss)
    
        data = pd.concat([LowCarb,Weightloss],axis=1)
        data.columns = ['Low Carb','Weight']
    return data
DataConfound1 = monte_carlo(100)
print(DataConfound1.head())
DataConfound2 = monte_carlo(100)
print (DataConfound2.head())

   Low Carb  Weight
0         0  180.90
1         0  212.96
2         0  144.45
3         1  203.77
4         0  202.01
   Low Carb  Weight
0         0  131.03
1         1  132.51
2         1  159.05
3         0  197.04
4         1  256.17


RMSE calculation

In [47]:
actual = DataConfound1
pred = DataConfound2
RMSE = sqrt(mse(actual,pred))
print(RMSE)

62.593902782140056


Treatment Effect

In [70]:
# Run OLS regression on DataConfound1
ols = smf.ols(formula="Weight ~ LowCarb",data=DataConfound1).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 13 Apr 2022   Prob (F-statistic):                nan
Time:                        11:24:09   Log-Likelihood:                -561.08
No. Observations:                 100   AIC:                             1124.
Df Residuals:                      99   BIC:                             1127.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     86.3048      3.324     25.964      0.0

#### Bias

In [71]:
Predicted_Weight = ols.predict()
Actual_Weight = DataConfound1["Weight"]
Bias = mean(Predicted_Weight) - mean(Actual_Weight)
print(Bias)

-2.842170943040401e-14


# 3. Selection Bias

a low carb diet is randomly assigned to patients as in (1), but the patients are only those that are deemed as obese. again, we are seeing the low carb diet effect on patient weight.

again, a coinflip decides whether a patient is to be put on a low carb diet, and the only patients are those that are obese

weight = a0 + a1LowCarb + a2Obese + e

### DAG

In [91]:
g = gr.Digraph()
g.edge("lowcarb", "obese")
g.edge("lowcarb","weight")
g.edge("obese","weight")
#g

### DGP

in this dataframe, the low carb diet assignment will only consist of 1's because everyone in this section has been assigned the treatment because they have been diagonosed with obesity (another column now in the dataframe consisting of only 1's). There is also a column for weight

In [50]:
n=100
np.random.seed(1234)

#Treatment;dummy only taking 1 since everyone in this set has been assigned treatment 
lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=1,1,1)} 

#Selection Bias;dummy only taking 1 since all patients are obese that are receiving the treatment
obese = {'Obese':np.where(np.random.normal(1,10,n)<=1,1,1)} 

weightloss = {'Weight':np.random.normal(230,10,n).round(2)} #outcome

Obese = pd.DataFrame(data=obese)
LowCarb = pd.DataFrame(data=lowcarb)
Weightloss = pd.DataFrame(data=weightloss)

df3 = pd.concat([LowCarb,Obese,Weightloss],axis=1)
df3 #our data for this

,Low Carb,Obese,Weight
0,1,1,226.80
1,1,1,223.80
2,1,1,231.57
3,1,1,224.29
4,1,1,240.58
...,...,...,...
95,1,1,243.43
96,1,1,229.49
97,1,1,226.36
98,1,1,214.47


### Monte Carlo Sim

In [51]:
data = []

def monte_carlo(n):
    for i in range(n):
        lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=1,1,1)} 

        obese = {'Obese':np.where(np.random.normal(1,10,n)<=1,1,1)} 
        
        weightloss = {'Weight':np.random.normal(230,10,n).round(2)}
        
        Obese = pd.DataFrame(data=obese)
        LowCarb = pd.DataFrame(data=lowcarb)
        Weightloss = pd.DataFrame(data=weightloss)
    
        data = pd.concat([LowCarb,Obese,Weightloss],axis=1)
    return data
DataSB1 = monte_carlo(100)
print(DataSB1.head())
DataSB2 = monte_carlo(100)
print (DataSB2.head())

   Low Carb  Obese  Weight
0         1      1  220.75
1         1      1  231.40
2         1      1  219.03
3         1      1  243.92
4         1      1  219.13
   Low Carb  Obese  Weight
0         1      1  234.04
1         1      1  233.38
2         1      1  236.41
3         1      1  213.42
4         1      1  226.79


RMSE Calculation

In [ ]:
actual = DataSB1
pred = DataSB2
RMSE = sqrt(mse(actual,pred))
print(RMSE)

Treatment Effect

In [52]:
# Run OLS regression on DataSB1
ols = smf.ols(formula="Weight ~ LowCarb + Obese",data=DataSB1).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 13 Apr 2022   Prob (F-statistic):                nan
Time:                        11:18:06   Log-Likelihood:                -363.69
No. Observations:                 100   AIC:                             729.4
Df Residuals:                      99   BIC:                             732.0
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     76.2552      0.308    247.726      0.0

#### Bias

In [69]:
Predicted_Weight = ols.predict()
Actual_Weight = DataSB1["Weight"]
Bias = mean(Predicted_Weight) - mean(Actual_Weight)
print(Bias)

-2.842170943040401e-14


### Uncontrolled

#### DAG

In [92]:
g = gr.Digraph()
g.edge("lowcarb","weight")
#g

#### DGP

no longer controlling for patients diagnosed with obesity, the Obese column is no longer included, but weight and low carb assinment remains (consisting of only 1's). 

In [54]:
n=100
np.random.seed(1234)

#Treatment;dummy only taking 1 since everyone in this set has been assigned treatment 
lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=1,1,1)} 

weightloss = {'Weight':np.random.normal(230,10,n).round(2)} #outcome

LowCarb = pd.DataFrame(data=lowcarb)
Weightloss = pd.DataFrame(data=weightloss)

df3 = pd.concat([LowCarb,Weightloss],axis=1)
df3 #our data for this

,Low Carb,Weight
0,1,232.91
1,1,235.67
2,1,235.04
3,1,232.85
4,1,234.84
...,...,...
95,1,225.68
96,1,228.39
97,1,238.89
98,1,232.88


#### Monte Carlo

In [55]:
data = []

def monte_carlo(n):
    for i in range(n):
        lowcarb = {'Low Carb':np.where(np.random.normal(1,10,n)<=1,1,1)} 

        weightloss = {'Weight':np.random.normal(230,10,n).round(2)}
        
        LowCarb = pd.DataFrame(data=lowcarb)
        Weightloss = pd.DataFrame(data=weightloss)
    
        data = pd.concat([LowCarb,Weightloss],axis=1)
    return data
DataSB1 = monte_carlo(100)
print(DataSB1.head())
DataSB2 = monte_carlo(100)
print (DataSB2.head())

   Low Carb  Weight
0         1  230.14
1         1  235.07
2         1  224.53
3         1  233.66
4         1  233.39
   Low Carb  Weight
0         1  222.47
1         1  222.69
2         1  226.78
3         1  232.62
4         1  241.72


RMSE calculation

In [ ]:
actual = DataSB1
pred = DataSB2
RMSE = sqrt(mse(actual,pred))
print(RMSE)

Treatment Effect

In [62]:
# Run OLS regression on DataSB1
ols = smf.ols(formula="Weight ~ LowCarb",data=DataSB1).fit()
print(ols.summary())

                            OLS Regression Results                            
Dep. Variable:                 Weight   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 13 Apr 2022   Prob (F-statistic):                nan
Time:                        11:20:06   Log-Likelihood:                -373.90
No. Observations:                 100   AIC:                             749.8
Df Residuals:                      99   BIC:                             752.4
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    114.4315      0.511    223.773      0.0

#### Bias

In [68]:
Predicted_Weight = ols.predict()
Actual_Weight = DataSB1["Weight"]
Bias = mean(Predicted_Weight) - mean(Actual_Weight) 
print(Bias)

-2.842170943040401e-14
